In [ ]:
%matplotlib notebook
%load_ext rpy2.ipython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
readRDS = robjects.r['readRDS']


def plot_benchmarks(value, edges_nodes, ilp_solvers, **kwargs):
    ni = 3 if "num_inputs" not in kwargs else kwargs["num_inputs"]
    nm = 3 if "num_meas" not in kwargs else kwargs["num_meas"]
    net_type = "Erdos" if "net_type" not in kwargs else kwargs["net_type"]
    figsize = (12, 5) if "figsize" not in kwargs else kwargs["figsize"]
    
    get_benchmark = lambda x, y, z : pd.read_csv(f"Output/{net_type}/E{x}_N{y}_I{ni}_M{nm}_S1_P2_2/{z}/benchmark.tsv", sep="\t")
    
    data = np.array([[get_benchmark(x, y, z)[value].values for x, y in edges_nodes] for z in ilp_solvers])
    
    if "scale" in kwargs: data = kwargs["scale"](data)
    x_ticklabels = [x[0] for x in edges_nodes]
    x_label = "Number of edges"
        
    fig, axs = plt.subplots(1, len(ilp_solvers), figsize=figsize, sharey=True)
    fig.suptitle(net_type)
    fig.subplots_adjust(bottom=0.2, left=0.1, right=0.9)
    for i, ax in enumerate(axs):
        ax.boxplot(data[i].tolist())
        ax.set_xticklabels(x_ticklabels)
        ax.set_xlabel(x_label)
        ax.set_title(ilp_solvers[i])
        
    if "ylabel" in kwargs:
        axs[0].set_ylabel(kwargs["ylabel"])
    if "file" in kwargs:
        fig.savefig(kwargs["file"])

        
time_settings = {"ilp_solvers": ["cplex", "cbc", "lpSolve"], 
                 "ylabel": "Execution time [mins]", 
                 "scale": lambda x : x/60,}
mem_settings = {"ilp_solvers": ["cplex", "cbc", "lpSolve"], 
                "ylabel": "Execution time [mins]", 
                "scale": lambda x : x/60,}
        
t_args1 = {**time_settings, 
           "edges_nodes": [[x, 50] for x in np.arange(200, 1000, 100)], 
           "file": "Figures/small_networks_time.svg"}
mem_args1 =  {**mem_settings,
              "edges_nodes": [[x, 50] for x in np.arange(200, 1000, 100)], 
              "file": "Figures/small_networks_memory.svg"}

t_args2 = {**time_settings, 
           "edges_nodes": [[x, 200] for x in np.arange(1000, 6000, 1000)], 
           "file": "Figures/medium_networks_time.svg"}
mem_args2 =  {**mem_settings,
              "edges_nodes": [[x, 200] for x in np.arange(1000, 6000, 1000)], 
              "file": "Figures/medium_networks_memory.svg"}

# Smaller networks

In [ ]:
plot_benchmarks("s", **t_args1, net_type="Erdos")

In [ ]:
plot_benchmarks("s", **t_args1, net_type="Powerlaw")

In [ ]:
plot_benchmarks("max_rss", **mem_args1, net_type="Erdos")

In [ ]:
plot_benchmarks("max_rss", **mem_args1, net_type="Powerlaw")

# Medium networks

In [ ]:
plot_benchmarks("s", **t_args2, net_type="Erdos")

In [ ]:
plot_benchmarks("s", **t_args2, net_type="Powerlaw")

In [ ]:
plot_benchmarks("max_rss", **mem_args2, net_type="Erdos")

In [ ]:
plot_benchmarks("max_rss", **mem_args2, net_type="Powerlaw")

In [ ]:
%%R
print(readRDS("Output/Erdos/E100_N20_I3_M3_S1_P2_2/cplex/result.Rds")$weightedSIF)
print(readRDS("Output/Erdos/E100_N20_I3_M3_S1_P2_2/cbc/result.Rds")$weightedSIF)
print(readRDS("Output/Erdos/E100_N20_I3_M3_S1_P2_2/lpSolve/result.Rds")$weightedSIF)